In [1]:
# Import Libraries
import numpy as np
import pandas as pd
import string
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from nltk.corpus import stopwords
from sklearn import metrics
from collections import Counter
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.sequence import pad_sequences
import warnings

In [3]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:
stopwords_list = stopwords.words('arabic')
warnings.filterwarnings("ignore", category=FutureWarning) # Ignore warnings

# Importing the dataset

In [5]:
!pip install datasets
from datasets import load_dataset

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [6]:
emotion_dataset = load_dataset("emotone_ar")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/5.13k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/862k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/10065 [00:00<?, ? examples/s]

In [7]:
emotion_dataset

DatasetDict({
    train: Dataset({
        features: ['tweet', 'label'],
        num_rows: 10065
    })
})

# EDA (Exploratory Data Analysis)

In [8]:
# Format the dataset as pandas dataFrame
emotion_dataset.set_format(type="pandas")
train = emotion_dataset["train"][:]

In [9]:
train.head()

,tweet,label
0,الاوليمبياد الجايه هكون لسه ف الكليه ..,0
1,عجز الموازنه وصل ل93.7 % من الناتج المحلي يعني...,1
2,كتنا نيله ف حظنا الهباب xD,3
3,جميعنا نريد تحقيق اهدافنا لكن تونس تالقت في حر...,2
4,الاوليمبياد نظامها مختلف .. ومواعيد المونديال ...,0


In [10]:
# Describe each label
labels_dict = {0:'neutral', 1:'anger', 2:'joy', 3:'sadness', 4:'love', 5:'sympathy', 6:'surprise'}
train['description'] = train['label'].map(labels_dict)
train.head()

,tweet,label,description
0,الاوليمبياد الجايه هكون لسه ف الكليه ..,0,neutral
1,عجز الموازنه وصل ل93.7 % من الناتج المحلي يعني...,1,anger
2,كتنا نيله ف حظنا الهباب xD,3,sadness
3,جميعنا نريد تحقيق اهدافنا لكن تونس تالقت في حر...,2,joy
4,الاوليمبياد نظامها مختلف .. ومواعيد المونديال ...,0,neutral


In [11]:
train['description'].value_counts(normalize=True)

,proportion
description,
neutral,0.174983
anger,0.163016
joy,0.144615
sadness,0.141793
love,0.137729
sympathy,0.119892
surprise,0.117972


The data is balanced.

In [12]:
#Train and Test split
train_sentences, val_sentences , train_labels, val_labels = train_test_split(train.tweet, train.label, test_size=0.3)

#Machine Learning

In [13]:
# Logistic Regression Classifier
vec = CountVectorizer()
clf = LogisticRegression()
pipe = make_pipeline(vec, clf)
pipe.fit(train_sentences, train_labels);

In [14]:
def print_report(pipe, x_test, y_test):
    y_pred = pipe.predict(x_test)
    report = metrics.classification_report(y_test, y_pred)
    print(report)
    print("accuracy: {:0.3f}".format(metrics.accuracy_score(y_test, y_pred)))

In [15]:
print_report(pipe, val_sentences, val_labels)

              precision    recall  f1-score   support

           0       0.64      0.81      0.72       480
           1       0.64      0.61      0.63       443
           2       0.40      0.51      0.45       372
           3       0.47      0.36      0.41       389
           4       0.64      0.65      0.65       358
           5       0.86      0.78      0.82       329
           6       0.50      0.41      0.45       306
           7       0.97      0.87      0.92       343

    accuracy                           0.63      3020
   macro avg       0.64      0.63      0.63      3020
weighted avg       0.64      0.63      0.63      3020

accuracy: 0.631


In [16]:
vec = TfidfVectorizer(analyzer='char_wb', ngram_range=(2, 5), min_df=.01, max_df=.3)
clf = LinearSVC() #Linear Support Vector
pipe_tfidf = make_pipeline(vec, clf)
pipe_tfidf.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidfvectorizer',
                 TfidfVectorizer(analyzer='char_wb', max_df=0.3, min_df=0.01,
                                 ngram_range=(2, 5))),
                ('linearsvc', LinearSVC())])

In [17]:
print_report(pipe_tfidf, val_sentences, val_labels)

              precision    recall  f1-score   support

           0       0.65      0.79      0.71       480
           1       0.59      0.66      0.62       443
           2       0.50      0.46      0.48       372
           3       0.50      0.40      0.44       389
           4       0.69      0.72      0.70       358
           5       0.85      0.84      0.85       329
           6       0.62      0.52      0.56       306
           7       0.92      0.89      0.91       343

    accuracy                           0.66      3020
   macro avg       0.66      0.66      0.66      3020
weighted avg       0.66      0.66      0.66      3020

accuracy: 0.661


## Deep Learning

**Data Preprocessing**

In [18]:
def remove_stopwords(text):
    filtered_words = [word for word in text.split() if word not in stopwords_list]
    return " ".join(filtered_words)

def remove_punct(text):
    translator = str.maketrans("", "", string.punctuation)
    return text.translate(translator)

In [19]:
train_sentences = train_sentences.map(remove_stopwords)
train_sentences = train_sentences.map(remove_punct)

val_sentences = val_sentences.map(remove_stopwords)
val_sentences = val_sentences.map(remove_punct)

In [20]:
#Count the occurrences of each word in a text column
def counter_word(text_col):
    count = Counter()
    for text in text_col.values:
        for word in text.split():
            count[word] += 1
    return count

In [21]:
# Count the unique words
counter = counter_word(train_sentences)
num_unique_words = len(counter)
num_unique_words

30997

In [22]:
counter.most_common(5)

[('الاوليمبياد', 1821), ('علي', 938), ('الله', 762), ('ان', 751), ('انا', 660)]

In [23]:
#Convert to NumPy arrays
train_sentences = train_sentences.to_numpy()
train_labels = pd.get_dummies(train_labels).values # One-hot expression
val_sentences = val_sentences.to_numpy()
val_labels = pd.get_dummies(val_labels)

In [24]:
train_sentences.shape, val_sentences.shape

((7045,), (3020,))

In [25]:
train_labels.shape, val_labels.shape

((7045, 8), (3020, 8))

In [26]:
# Each word got unique index
tokenizer = Tokenizer(num_words=num_unique_words)
tokenizer.fit_on_texts(train_sentences)
word_index = tokenizer.word_index

In [27]:
# Apply on train and validation
train_sequences = tokenizer.texts_to_sequences(train_sentences)
val_sequences = tokenizer.texts_to_sequences(val_sentences)

In [28]:
# Pad the sequences to have the same length (to ensure compatibility with NN)
max_length = 30
train_padded = pad_sequences(train_sequences, maxlen=max_length, padding="post", truncating="post")
val_padded = pad_sequences(val_sequences, maxlen=max_length, padding="post", truncating="post")

In [29]:
#Check
train_padded.shape, val_padded.shape

((7045, 30), (3020, 30))

In [35]:
# Define model parameters
NUM_FILTERS = 64
NUM_WORDS = 4
embed_dim = 256
batch_size = 1024
EPOCHS = 25

# Build the model
model = keras.models.Sequential()
model.add(layers.Embedding(num_unique_words, embed_dim))
model.add(layers.SpatialDropout1D(0.6))
model.add(layers.SpatialDropout1D(0.4))
model.add(layers.SpatialDropout1D(0.2))
model.add(layers.Conv1D(filters=NUM_FILTERS, kernel_size=NUM_WORDS, activation="relu"))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(8, activation="softmax"))

# Compile the model
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["acc"])

# Train the model
history = model.fit(
    train_padded,
    train_labels,
    batch_size=batch_size,
    epochs=EPOCHS,
    validation_data=(val_padded, val_labels)
)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(val_padded, val_labels, verbose=1)

# Print test accuracy
print(f"Test Accuracy: {test_accuracy:.2f}")
print(model.summary())

Epoch 1/25
7/7 ━━━━━━━━━━━━━━━━━━━━ 7s 542ms/step - acc: 0.1302 - loss: 2.0927 - val_acc: 0.2139 - val_loss: 2.0616
Epoch 2/25
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - acc: 0.1565 - loss: 2.0591 - val_acc: 0.2825 - val_loss: 2.0367
Epoch 3/25
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - acc: 0.2266 - loss: 2.0283 - val_acc: 0.3361 - val_loss: 1.9956
Epoch 4/25
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - acc: 0.2753 - loss: 1.9830 - val_acc: 0.3709 - val_loss: 1.9180
Epoch 5/25
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - acc: 0.3335 - loss: 1.8981 - val_acc: 0.4119 - val_loss: 1.8052
Epoch 6/25
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - acc: 0.3984 - loss: 1.7873 - val_acc: 0.4685 - val_loss: 1.7021
Epoch 7/25
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - acc: 0.4662 - loss: 1.6797 - val_acc: 0.5026 - val_loss: 1.6274
Epoch 8/25
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - acc: 0.5294 - loss: 1.5661 - val_acc: 0.5325 - val_loss: 1.5631
Epoch 9/25
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - acc: 0.5516 - loss: 1.4900 -

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_4 (Embedding)              │ (None, 30, 256)             │       7,935,232 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ spatial_dropout1d_12                 │ (None, 30, 256)             │               0 │
│ (SpatialDropout1D)                   │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ spatial_dropout1d_13                 │ (None, 30, 256)             │               0 │
│ (SpatialDropout1D)                   │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ spatial_dropout1d_14                 │ (None, 30, 256)             │               0 │
│ (SpatialDropout1D)                   │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_4 (Conv1D)                    │ (None, 27, 64)              │          65,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_max_pooling1d_4               │ (None, 64)                  │               0 │
│ (GlobalMaxPooling1D)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 8)                   │             520 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 24,004,058 (91.57 MB)

 Trainable params: 8,001,352 (30.52 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 16,002,706 (61.05 MB)

None
